In [5]:

import random
import matplotlib.pyplot as plt
from keras.models import Model ,load_model
from keras.layers import * 
import cv2
import numpy as np
import time


def play_game(choice):
    if(choice==0):
        return "You : Scissors"
    elif(choice==1):
        return "You : stone"
    elif(choice==2):
        return "You : paper"

def winner_game(choice):
    choices=[0,1,2]
    comp=random.choice(choices)
    if(comp==0):
        if(choice==1):
            return "Comp : Scissors ,You Won"
        elif(choice==2):
            return "Comp : Scissors ,You Lost"
        else:
            return "Comp : Scissors ,Draw"
    elif(comp==1):
        if(choice==0):
            return "Comp : Stone ,You Lost"
        elif(choice==2):
            return "Comp : Stone ,You Won"
        else:
            return "Comp : Stone ,Draw"
    if(comp==2):
        if(choice==1):
            return "Comp : Paper ,You Lost"
        elif(choice==0):
            return "Comp : Paper ,You Won"
        else:
            return "Comp : Paper ,Draw"

In [6]:


new_model=load_model('Stone_Paper_Scissors.h5')
past=time.time()
cap = cv2.VideoCapture(0)
score_comp=0
score_you=0
text="Nothing"
flag=0
while(True):
   
    ret, frame = cap.read()
  
    frame=cv2.flip(frame,1)
    roi=frame[10:350,320:660]
    
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1
    color = (255, 0, 0)
    thickness = cv2.FILLED
    margin = 2
     
    cv2.rectangle(frame,(340,10),(620,350),(0,255,0),2)
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    cv2.putText(frame,"Scores:", (340,380), font_face, scale, (255,0,0), 2, cv2.LINE_AA)
    cv2.putText(frame,"You:", (340,420), font_face, scale, (0,0,255), 2, cv2.LINE_AA)
    cv2.putText(frame,str(score_you), (510,420), font_face, scale, (0,0,255), 2, cv2.LINE_AA)
    cv2.putText(frame,"Comp:", (340,460), font_face, scale, (0,255,255), 2, cv2.LINE_AA)
    cv2.putText(frame,str(score_comp), (510,460), font_face, scale, (0,255,255), 2, cv2.LINE_AA)
        
    
    
    cv2.imshow('original',frame)
    k =cv2.waitKey(1)
    
    if (time.time()-past>10):
        past=time.time()
        flag=0
    elif(time.time()-past>5):
        cv2.putText(hsv,"Scissor", (30,300), font_face, scale, color, 2, cv2.LINE_AA)
        
        
        if(time.time()-past>7):
            img = cv2.resize(hsv, (320, 120))
            img = np.array(img, dtype="uint8")
            img = img.reshape(1,120, 320, 1)
            predictions = new_model.predict(img)
            text=play_game(np.argmax(predictions))
            if flag==0:
                winner=winner_game(np.argmax(predictions))
                if winner.find('Won')!=-1:
                    score_you=score_you+1
                elif winner.find('Lost')!=-1:
                    score_comp=score_comp+1
                flag=1
            cv2.putText(hsv, text, (30,30), font_face, scale, (0,0,255), 2, cv2.LINE_AA)
            cv2.putText(hsv, winner, (30,150), font_face, 0.6,(0,255,255), 2, cv2.LINE_AA)
        
    elif(time.time()-past>3):
        cv2.putText(hsv,"Paper", (30,300), font_face, scale, color, 2, cv2.LINE_AA)
    elif(time.time()-past>1):
        cv2.putText(hsv,"Stone", (30,300), font_face, scale, color, 2, cv2.LINE_AA)
        
    
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
     
        
    cv2.imshow('mask',hsv)
#When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Escape hit, closing...
